# Oscar Prediction 2020 Part I - Web Scraping

In order to predict the 
We did some preparation

## 1. Web Scraping Awards Result

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import requests as rq
from datetime import datetime

In [3]:
pd.options.display.max_rows = 168
pd.options.display.max_columns = 23

In [28]:
browser = webdriver.Chrome(executable_path="D:/chromedriver.exe")

In [ ]:
award_show_list = pd.read_csv("E:/Downloads/award_show_list.csv")
award_show_list

In [ ]:
awards_all = pd.DataFrame()

for award_show in range(len(award_show_list)):
    award_show_name = award_show_list['award_show'][award_show]
    award_show_url = award_show_list['url'][award_show]
    
    browser.get(award_show_url)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    
    find = soup.find('div', {'class': 'event-history-widget'}).find_all('a')
    award_years = [int(ele.text.strip()) for ele in find]
    
    award_urls = ['https://www.imdb.com/'+ele.get('href') for ele in find]
    
    award_dict = {"award_years" : award_years, "final_urls" : award_urls}
    award_url_df = pd.DataFrame(award_dict)
    ref_year = 1
    row_num = 1
    
    while ref_year <= 25 and row_num <= len(award_url_df):
        url = award_url_df['final_urls'][row_num-1]
        year = award_url_df['award_years'][row_num-1]
        browser.get(url)
        soup = BeautifulSoup(browser.page_source, 'lxml')
        number_of_awards = len(soup.find_all('div', {'class': 'event-widgets__award-category'})) - len(soup.find_all('div', {'class': 'event-widgets__award-name'})) + 1
        if number_of_awards > 1:
            for award in range(number_of_awards):
                award_name = soup.findAll('div', {'class': 'event-widgets__award-category-name'})[award]
                award_name = award_name.text.strip()
                
                nominees = soup.findAll('div', {'class': 'event-widgets__award-category'})[award].find_all('div', {'class': 'event-widgets__primary-nominees'})
                nominees = [ele.text.strip() for ele in nominees]
                
                imdb_urls =[]
                for n in range(len(soup.findAll('div', {'class': 'event-widgets__award-category'})[award].findAll('div', {'class': 'event-widgets__primary-nominees'}))):
                    try:
                        imdb_urls.append('https://www.imdb.com/'+soup.findAll('div', {'class': 'event-widgets__award-category'})[award].findAll('div', {'class': 'event-widgets__primary-nominees'})[n].find('a').get('href'))
                    except:
                        imdb_urls.append(None)
                details = soup.findAll('div', {'class': 'event-widgets__award-category'})[award].find_all('div', {'class': 'event-widgets__secondary-nominees'})
                details = [ele.text.strip() for ele in details]
                
                temp_df = {"nominees" : nominees,"details" : details,"imdb_urls":imdb_urls}
                temp_df = pd.DataFrame(temp_df)
                temp_df['award_show'] = award_show_name
                temp_df['year'] = year
                temp_df['award'] = award_name
                temp_df['winner'] = [1 if index == 0 else 0 for index in temp_df.index]
                temp_df['ref_year'] = ref_year
                
                awards_all = pd.concat([awards_all, temp_df])
            ref_year += 1 
            row_num += 1
        else:
            row_num += 1

In [ ]:
awards_all

In [3]:
#delete
awards_all = pd.read_csv('2.csv')

## 2. Web Scraping Movie Features

In [ ]:
oscar = awards_all[awards_all['award_show'] == 'Academy Awards']
oscar_picture = oscar[oscar['award'].isin(['Best Motion Picture of the Year','Best Picture'])]
oscar_picture = oscar_picture
oscar_picture

In [ ]:
oscar_picture = oscar_picture.reset_index(drop=True)
metabase1 = []
rating1 = []
popularity1 = []
genres1 = []
budget1 = []
gross1 = []
minute1 = []

#for row_num in range(5):
for row_num in range(len(oscar_picture)):
    imdb_url = oscar_picture['imdb_urls'][row_num]
    browser.get(imdb_url)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    
    metabase = soup.find('div', {'class': 'titleReviewBarItem'})
    metabase = metabase.text.strip()
    pattern = re.compile(r'(\d+,?)+')
    metabase = pattern.findall(re.sub(',', "", metabase))
    metabase1.append(metabase[0])
    
    rating = soup.find('div', {'class': 'ratingValue'}).text.strip()
    rating = re.sub(r'/.*$', "", rating)
    rating1.append(rating)
    
    try:
        popularity = soup.findAll('span', {'class': 'subText'})[2].text.strip()
        pattern4 = re.compile(r'(\d+)\n')
        if '\n' in popularity:
            popularity = pattern4.findall(popularity)[0]
        popularity1.append(popularity)
    except:
        popularity1.append(None)
        
    genres =[]
    for n in range(len(soup.findAll('div', {'class': 'see-more inline canwrap'})[1].find_all('a'))):
        genres.append(soup.findAll('div', {'class': 'see-more inline canwrap'})[1].findAll('a')[n].text.strip())
    genres1.append(genres)
    
    budget = soup.find('div', {'id': 'titleDetails'}).findAll('div', {'class': 'txt-block'})[6].text.strip()
    pattern1 = re.compile(r'(\d+)+\n')
    try:
        budget = pattern1.findall(re.sub(',', "", budget))[0]
        budget1.append(budget)
    except:
        budget1.append(None)
    
    gross = soup.find('div', {'id': 'titleDetails'}).text.strip()
    pattern2 = re.compile(r'Cumulative Worldwide Gross\:.*\$(\d+)')
    gross = pattern2.findall(re.sub(',', "", gross))[0]
    gross1.append(gross)
    
    minute = soup.find('div', {'id': 'titleDetails'}).text.strip()
    pattern3 = re.compile(r'(\d+).*min')
    try:
        minute = pattern3.findall(minute)[0]
        minute1.append(minute)
    except:
        minute1.append(None)

oscar_picture = oscar_picture.assign(metabase = metabase1,rating = rating1, popularity = popularity1, genres = genres1, budget = budget1, gross = gross1, minute = minute1)

In [ ]:
oscar_picture

## 2. Create Other Movie Dataset

In [ ]:
other = awards_all[awards_all['award_show'] != 'Academy Awards']

other_picture = other[other['award'].isin(["Best Edited Feature Film", 
                  "Best Edited Feature Film - Dramatic", 
                  "Best Edited Feature Film - Comedy or Musical",
                  "Best Edited Feature Film - Comedy", 
                  "Best Film", 
                  "Best Picture", "Best Motion Picture, Comedy or Musical",
                  "Best Motion Picture, Drama", "Best Motion Picture - Drama", 
                  "Best Motion Picture - Musical or Comedy",
                  "Best Motion Picture - Comedy or Musical", "Best Motion Picture"])][['nominees','award_show','year','winner']]

In [ ]:
all_picture= pd.merge(oscar_picture,other_picture,on=['nominees','year'],how='left')
all_picture1 = all_picture.pivot(index='nominees', columns='award_show_y', values='winner_y').fillna(-1)
all_picture2 = pd.merge(oscar_picture,all_picture1,on=['nominees'],how='left')

In [5]:
#delete later
oscar_picture=pd.read_csv('3.csv')
oscar_picture

,Unnamed: 0,Unnamed: 0.1,nominees,details,imdb_urls,award_show,year,award,winner,ref_year,metabase,rating,popularity,genres,budget,gross,minute,American Cinema Editors,BAFTA,Chicago Film Critics,Critics Choice,Golden Globes,Satellite
0,0,0,1917,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren, ...",https://www.imdb.com//title/tt8579674/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,1,1,78,8.5,1.0,"['Drama', 'War']",100000000.0,254142909,119,-1.0,1.0,-1.0,0.0,1.0,-1.0
1,1,1,Ford v Ferrari,"Peter Chernin, Jenno Topping, James Mangold",https://www.imdb.com//title/tt1950186/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,81,8.2,13.0,"['Action', 'Biography', 'Drama', 'Sport']",97600000.0,222277771,152,0.0,-1.0,-1.0,0.0,-1.0,-1.0
2,2,2,Jojo Rabbit,"Carthew Neal, Taika Waititi",https://www.imdb.com//title/tt2584384/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,58,8.0,8.0,"['Comedy', 'Drama', 'War']",349555.0,65647700,108,1.0,-1.0,-1.0,0.0,0.0,-1.0
3,3,3,Joker,"Todd Phillips, Bradley Cooper, Emma Tillinger ...",https://www.imdb.com//title/tt7286456/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,59,8.6,9.0,"['Crime', 'Drama', 'Thriller']",55000000.0,1071884607,122,0.0,0.0,-1.0,0.0,0.0,-1.0
4,4,4,Little Women,Amy Pascal,https://www.imdb.com//title/tt3281548/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,91,8.1,11.0,"['Drama', 'Romance']",40000000.0,164959722,135,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
5,5,5,Marriage Story,"Noah Baumbach, David Heyman",https://www.imdb.com//title/tt7653254/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,93,8.1,23.0,"['Comedy', 'Drama', 'Romance']",18600000.0,323382,137,0.0,-1.0,-1.0,0.0,0.0,-1.0
6,6,6,Once Upon a Time... in Hollywood,"David Heyman, Shannon McIntosh, Quentin Tarantino",https://www.imdb.com//title/tt7131622/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,83,7.7,10.0,"['Comedy', 'Drama']",90000000.0,374035441,161,0.0,0.0,-1.0,1.0,1.0,-1.0
7,7,7,Parasite,"Sin-ae Kwak, Bong Joon Ho",https://www.imdb.com//title/tt6751668/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,96,8.6,3.0,"['Comedy', 'Crime', 'Drama', 'Thriller']",393216.0,163862304,132,1.0,0.0,-1.0,0.0,-1.0,-1.0
8,8,8,The Irishman,"Martin Scorsese, Robert De Niro, Jane Rosentha...",https://www.imdb.com//title/tt1302006/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,94,8.0,15.0,"['Biography', 'Crime', 'Drama']",159000000.0,961224,209,0.0,0.0,-1.0,0.0,0.0,-1.0
9,9,0,Green Book,"Jim Burke, Charles B. Wessler, Brian Hayes Cur...",https://www.imdb.com//title/tt6966692/?ref_=ev...,Academy Awards,2019,Best Motion Picture of the Year,1,2,69,8.2,111.0,"['Biography', 'Comedy', 'Drama', 'Music']",23000000.0,329717166,130,0.0,0.0,-1.0,0.0,1.0,-1.0


## 4. Web Scraping Critic Review Scores

### 4.1 Get Oscar Dates

In [6]:
soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/List_of_Academy_Awards_ceremonies').text, 'lxml')

In [7]:
table = soup.find('table', {'class': 'wikitable'}).find('tbody')
table = table.findNext('table', {'class': 'wikitable'}).find('tbody')

In [8]:
data = []
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [9]:
dates = [x[1] for x in data[1:] if len(x) >= 2]
dates1=[datetime.strptime(ele, '%B %d, %Y').year for ele in dates]

In [10]:
a=[dates,dates1]
data=pd.DataFrame(a)
data=data.T
data.rename(columns={0:'date',1:'year'},inplace=True)

In [11]:
oscar_picture1 = pd.merge(oscar_picture,data,how='left')
oscar_picture1

,Unnamed: 0,Unnamed: 0.1,nominees,details,imdb_urls,award_show,year,award,winner,ref_year,metabase,...,genres,budget,gross,minute,American Cinema Editors,BAFTA,Chicago Film Critics,Critics Choice,Golden Globes,Satellite,date
0,0,0,1917,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren, ...",https://www.imdb.com//title/tt8579674/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,1,1,78,...,"['Drama', 'War']",100000000.0,254142909,119,-1.0,1.0,-1.0,0.0,1.0,-1.0,"February 9, 2020"
1,1,1,Ford v Ferrari,"Peter Chernin, Jenno Topping, James Mangold",https://www.imdb.com//title/tt1950186/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,81,...,"['Action', 'Biography', 'Drama', 'Sport']",97600000.0,222277771,152,0.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020"
2,2,2,Jojo Rabbit,"Carthew Neal, Taika Waititi",https://www.imdb.com//title/tt2584384/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,58,...,"['Comedy', 'Drama', 'War']",349555.0,65647700,108,1.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020"
3,3,3,Joker,"Todd Phillips, Bradley Cooper, Emma Tillinger ...",https://www.imdb.com//title/tt7286456/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,59,...,"['Crime', 'Drama', 'Thriller']",55000000.0,1071884607,122,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020"
4,4,4,Little Women,Amy Pascal,https://www.imdb.com//title/tt3281548/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,91,...,"['Drama', 'Romance']",40000000.0,164959722,135,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020"
5,5,5,Marriage Story,"Noah Baumbach, David Heyman",https://www.imdb.com//title/tt7653254/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,93,...,"['Comedy', 'Drama', 'Romance']",18600000.0,323382,137,0.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020"
6,6,6,Once Upon a Time... in Hollywood,"David Heyman, Shannon McIntosh, Quentin Tarantino",https://www.imdb.com//title/tt7131622/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,83,...,"['Comedy', 'Drama']",90000000.0,374035441,161,0.0,0.0,-1.0,1.0,1.0,-1.0,"February 9, 2020"
7,7,7,Parasite,"Sin-ae Kwak, Bong Joon Ho",https://www.imdb.com//title/tt6751668/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,96,...,"['Comedy', 'Crime', 'Drama', 'Thriller']",393216.0,163862304,132,1.0,0.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020"
8,8,8,The Irishman,"Martin Scorsese, Robert De Niro, Jane Rosentha...",https://www.imdb.com//title/tt1302006/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,94,...,"['Biography', 'Crime', 'Drama']",159000000.0,961224,209,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020"
9,9,0,Green Book,"Jim Burke, Charles B. Wessler, Brian Hayes Cur...",https://www.imdb.com//title/tt6966692/?ref_=ev...,Academy Awards,2019,Best Motion Picture of the Year,1,2,69,...,"['Biography', 'Comedy', 'Drama', 'Music']",23000000.0,329717166,130,0.0,0.0,-1.0,0.0,1.0,-1.0,"February 24, 2019"


### 4.2 Get Rotten Tomato Link

In [12]:
pattern4 = re.compile(r'(tt\d{7})')
oscar_picture['tt'] = 'test'
for n in range(len(oscar_picture1)):
    oscar_picture1.loc[n,'tt'] = pattern4.findall(oscar_picture.loc[n,'imdb_urls'])
oscar_picture1

,Unnamed: 0,Unnamed: 0.1,nominees,details,imdb_urls,award_show,year,award,winner,ref_year,metabase,...,budget,gross,minute,American Cinema Editors,BAFTA,Chicago Film Critics,Critics Choice,Golden Globes,Satellite,date,tt
0,0,0,1917,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren, ...",https://www.imdb.com//title/tt8579674/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,1,1,78,...,100000000.0,254142909,119,-1.0,1.0,-1.0,0.0,1.0,-1.0,"February 9, 2020",tt8579674
1,1,1,Ford v Ferrari,"Peter Chernin, Jenno Topping, James Mangold",https://www.imdb.com//title/tt1950186/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,81,...,97600000.0,222277771,152,0.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt1950186
2,2,2,Jojo Rabbit,"Carthew Neal, Taika Waititi",https://www.imdb.com//title/tt2584384/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,58,...,349555.0,65647700,108,1.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt2584384
3,3,3,Joker,"Todd Phillips, Bradley Cooper, Emma Tillinger ...",https://www.imdb.com//title/tt7286456/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,59,...,55000000.0,1071884607,122,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt7286456
4,4,4,Little Women,Amy Pascal,https://www.imdb.com//title/tt3281548/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,91,...,40000000.0,164959722,135,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt3281548
5,5,5,Marriage Story,"Noah Baumbach, David Heyman",https://www.imdb.com//title/tt7653254/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,93,...,18600000.0,323382,137,0.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt7653254
6,6,6,Once Upon a Time... in Hollywood,"David Heyman, Shannon McIntosh, Quentin Tarantino",https://www.imdb.com//title/tt7131622/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,83,...,90000000.0,374035441,161,0.0,0.0,-1.0,1.0,1.0,-1.0,"February 9, 2020",tt7131622
7,7,7,Parasite,"Sin-ae Kwak, Bong Joon Ho",https://www.imdb.com//title/tt6751668/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,96,...,393216.0,163862304,132,1.0,0.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt6751668
8,8,8,The Irishman,"Martin Scorsese, Robert De Niro, Jane Rosentha...",https://www.imdb.com//title/tt1302006/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,94,...,159000000.0,961224,209,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt1302006
9,9,0,Green Book,"Jim Burke, Charles B. Wessler, Brian Hayes Cur...",https://www.imdb.com//title/tt6966692/?ref_=ev...,Academy Awards,2019,Best Motion Picture of the Year,1,2,69,...,23000000.0,329717166,130,0.0,0.0,-1.0,0.0,1.0,-1.0,"February 24, 2019",tt6966692


In [13]:
OMDB_API_KEY = '14631071'
tomatoURL = []

for n in range(len(oscar_picture1)):
    resp = rq.get('http://www.omdbapi.com/?i='+oscar_picture1.loc[n,'tt']+'&apikey='+OMDB_API_KEY+'&tomatoes=true')
    json = resp.json()
    tomatoURL.append(json['tomatoURL'])
oscar_picture1['tomatoURL']=pd.Series(tomatoURL)

In [35]:
oscar_picture1.iloc[1,-1] = 'https://www.rottentomatoes.com/m/ford_v_ferrari'
oscar_picture1.iloc[14,-1] = 'https://www.rottentomatoes.com/m/roma_2018'
oscar_picture1['tomatoURL']
#37 41 47 48 62 68 81 84

0             https://www.rottentomatoes.com/m/1917_2019
1        https://www.rottentomatoes.com/m/ford_v_ferrari
2           https://www.rottentomatoes.com/m/jojo_rabbit
3            https://www.rottentomatoes.com/m/joker_2019
4      https://www.rottentomatoes.com/m/little_women_...
5      https://www.rottentomatoes.com/m/marriage_stor...
6      https://www.rottentomatoes.com/m/once_upon_a_t...
7         https://www.rottentomatoes.com/m/parasite_2019
8      https://www.rottentomatoes.com/m/the_irishman_...
9            https://www.rottentomatoes.com/m/green_book
10       http://www.rottentomatoes.com/m/a_star_is_born/
11     https://www.rottentomatoes.com/m/black_klansma...
12     http://www.rottentomatoes.com/m/black_panther_...
13     https://www.rottentomatoes.com/m/bohemian_rhap...
14            https://www.rottentomatoes.com/m/roma_2018
15        https://www.rottentomatoes.com/m/the_favourite
16        https://www.rottentomatoes.com/m/backseat_2018
17     https://www.rottentomato

### 4.3 Get Critic Review Scores

In [62]:
test = 'https://www.rottentomatoes.com/m/1917_2019'

review_url =  test+'/reviews'

In [63]:
#for test
browser.get(review_url)
time.sleep(1)
soup = BeautifulSoup(browser.page_source, 'lxml')
page_section = soup.find('span', {'class': 'pageInfo'})
next_page = page_section.findNext('a', {'class': 'btn btn-xs btn-primary-rt'})['href']

score_list = []
date_list = []

while next_page is not None:
    browser.get(review_url)
    time.sleep(1)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    getPageSocre(review_url)
    page_section = soup.find('span', {'class': 'pageInfo'})
    if len(soup.findAll('a', {'class': 'btn btn-xs btn-primary-rt'})) != 4 and review_url !=  test+'/reviews':
        next_page = None
    else:
        next_page = page_section.findNext('a', {'class': 'btn btn-xs btn-primary-rt'})['href']
        review_url = base_url + next_page
    
score_list1=[]
for i in range(len(score_list)):
    if datetime.strptime(date_list[i], '%B %d, %Y') < datetime.strptime('February 9, 2020', '%B %d, %Y'):
        score_list1.append(float(score_list[i].split('/')[0])/float(score_list[i].split('/')[1]))
try:
    avg_score.append(sum([i for i in score_list1])/len(score_list1))
except:
    avg_score.append(None)

In [64]:
avg_score

[0.8389189189189186,
 0.7880454545454547,
 0.7585294117647061,
 0.7386500000000001,
 0.8435714285714284,
 0.8511864406779657,
 0.7941903357425745,
 0.9061546862745097,
 0.8641666666666666,
 0.7327272727272726,
 0.8180317321333335,
 0.8339062500000001,
 0.8279920512500002,
 0.6165909090909089,
 0.8389189189189186,
 0.7880454545454547,
 0.7585294117647061,
 0.7386500000000001,
 0.8435714285714284,
 0.8511864406779657,
 0.7941903357425745,
 0.9061546862745097,
 0.8641666666666666,
 0.7327272727272726,
 0.8180317321333335,
 0.8339062500000001,
 0.8279920512500002,
 0.6165909090909089,
 0.8680434782608691,
 0.8525,
 0.645873015873016,
 0.8496710526315788,
 0.8271428571428568,
 0.7490217391304348,
 0.8194805194805194,
 0.8494545454545455,
 0.8509803921568627,
 0.7946874999999998,
 0.8070089285714285,
 0.8632666666666665,
 0.8530821917808218,
 0.7909523809523807,
 0.7357608695652174,
 0.8368085106382979,
 0.7646999999999999,
 0.8479999999999998,
 0.7639705882352941,
 0.8474999999999998,
 0.86

In [15]:
def getPageSocre(review_url):
    reviews = soup.find('div', {'class': 'review_table'})
    cur = reviews.findNext('div', {'class': 'review-link'})
    date = reviews.findNext('div', {'class': 'review-date'})
    while cur is not None and date is not None:
        if pattern6.findall(cur.text.strip()) is not None:
            try:
                score_list.append(pattern6.findall(cur.text.strip())[0])
                date_list.append(date.text.strip())
                cur = cur.findNext('div', {'class': 'review-link'})
                date = date.findNext('div', {'class': 'review-date'})
            except:
                cur = cur.findNext('div', {'class': 'review-link'})
                date = date.findNext('div', {'class': 'review-date'})

In [29]:
import time
pattern6 = re.compile(r'(\d+\.?\d+?\/\d+)')
base_url = 'https://www.rottentomatoes.com/'
avg_score = []


In [36]:
for n in range(len(oscar_picture1)):
    url = oscar_picture1['tomatoURL'][n]
    review_url =  url+'/reviews'
    try:
        browser.get(review_url)
        time.sleep(5)
        soup = BeautifulSoup(browser.page_source, 'lxml')
        page_section = soup.find('span', {'class': 'pageInfo'})
        next_page = page_section.findNext('a', {'class': 'btn btn-xs btn-primary-rt'})['href']

        score_list = []
        date_list = []

        while next_page is not None:
            browser.get(review_url)
            time.sleep(5)
            soup = BeautifulSoup(browser.page_source, 'lxml')
            getPageSocre(review_url)
            page_section = soup.find('span', {'class': 'pageInfo'})
            if len(soup.findAll('a', {'class': 'btn btn-xs btn-primary-rt'})) != 4 and review_url !=  url+'/reviews':
                next_page = None
            else:
                next_page = page_section.findNext('a', {'class': 'btn btn-xs btn-primary-rt'})['href']
                review_url = base_url + next_page
    
        score_list1=[]
        for i in range(len(score_list)):
            if datetime.strptime(date_list[i], '%B %d, %Y') < datetime.strptime(oscar_picture1['date'][n], '%B %d, %Y'):
                score_list1.append(float(score_list[i].split('/')[0])/float(score_list[i].split('/')[1]))
        try:
            avg_score.append(sum([i for i in score_list1])/len(score_list1))
        except:
            avg_score.append(None)
    except:
        avg_score.append(None)

In [69]:
avg_score1= avg_score[14:175]

In [72]:
oscar_picture1['avg_score']=pd.Series(avg_score1)
oscar_picture1

,Unnamed: 0,Unnamed: 0.1,nominees,details,imdb_urls,award_show,year,award,winner,ref_year,metabase,...,minute,American Cinema Editors,BAFTA,Chicago Film Critics,Critics Choice,Golden Globes,Satellite,date,tt,tomatoURL,avg_score
0,0,0,1917,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren, ...",https://www.imdb.com//title/tt8579674/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,1,1,78,...,119,-1.0,1.0,-1.0,0.0,1.0,-1.0,"February 9, 2020",tt8579674,https://www.rottentomatoes.com/m/1917_2019,0.838919
1,1,1,Ford v Ferrari,"Peter Chernin, Jenno Topping, James Mangold",https://www.imdb.com//title/tt1950186/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,81,...,152,0.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt1950186,https://www.rottentomatoes.com/m/ford_v_ferrari,0.788045
2,2,2,Jojo Rabbit,"Carthew Neal, Taika Waititi",https://www.imdb.com//title/tt2584384/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,58,...,108,1.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt2584384,https://www.rottentomatoes.com/m/jojo_rabbit,0.758529
3,3,3,Joker,"Todd Phillips, Bradley Cooper, Emma Tillinger ...",https://www.imdb.com//title/tt7286456/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,59,...,122,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt7286456,https://www.rottentomatoes.com/m/joker_2019,0.738650
4,4,4,Little Women,Amy Pascal,https://www.imdb.com//title/tt3281548/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,91,...,135,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt3281548,https://www.rottentomatoes.com/m/little_women_...,0.843571
5,5,5,Marriage Story,"Noah Baumbach, David Heyman",https://www.imdb.com//title/tt7653254/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,93,...,137,0.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt7653254,https://www.rottentomatoes.com/m/marriage_stor...,0.851186
6,6,6,Once Upon a Time... in Hollywood,"David Heyman, Shannon McIntosh, Quentin Tarantino",https://www.imdb.com//title/tt7131622/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,83,...,161,0.0,0.0,-1.0,1.0,1.0,-1.0,"February 9, 2020",tt7131622,https://www.rottentomatoes.com/m/once_upon_a_t...,0.794190
7,7,7,Parasite,"Sin-ae Kwak, Bong Joon Ho",https://www.imdb.com//title/tt6751668/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,96,...,132,1.0,0.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt6751668,https://www.rottentomatoes.com/m/parasite_2019,0.906155
8,8,8,The Irishman,"Martin Scorsese, Robert De Niro, Jane Rosentha...",https://www.imdb.com//title/tt1302006/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,94,...,209,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt1302006,https://www.rottentomatoes.com/m/the_irishman_...,0.864167
9,9,0,Green Book,"Jim Burke, Charles B. Wessler, Brian Hayes Cur...",https://www.imdb.com//title/tt6966692/?ref_=ev...,Academy Awards,2019,Best Motion Picture of the Year,1,2,69,...,130,0.0,0.0,-1.0,0.0,1.0,-1.0,"February 24, 2019",tt6966692,https://www.rottentomatoes.com/m/green_book,0.732727


In [79]:
oscar_picture1=oscar_picture1.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
oscar_picture1

,nominees,details,imdb_urls,award_show,year,award,winner,ref_year,metabase,rating,popularity,...,minute,American Cinema Editors,BAFTA,Chicago Film Critics,Critics Choice,Golden Globes,Satellite,date,tt,tomatoURL,avg_score
0,1917,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren, ...",https://www.imdb.com//title/tt8579674/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,1,1,78,8.5,1.0,...,119,-1.0,1.0,-1.0,0.0,1.0,-1.0,"February 9, 2020",tt8579674,https://www.rottentomatoes.com/m/1917_2019,0.838919
1,Ford v Ferrari,"Peter Chernin, Jenno Topping, James Mangold",https://www.imdb.com//title/tt1950186/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,81,8.2,13.0,...,152,0.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt1950186,https://www.rottentomatoes.com/m/ford_v_ferrari,0.788045
2,Jojo Rabbit,"Carthew Neal, Taika Waititi",https://www.imdb.com//title/tt2584384/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,58,8.0,8.0,...,108,1.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt2584384,https://www.rottentomatoes.com/m/jojo_rabbit,0.758529
3,Joker,"Todd Phillips, Bradley Cooper, Emma Tillinger ...",https://www.imdb.com//title/tt7286456/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,59,8.6,9.0,...,122,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt7286456,https://www.rottentomatoes.com/m/joker_2019,0.738650
4,Little Women,Amy Pascal,https://www.imdb.com//title/tt3281548/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,91,8.1,11.0,...,135,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt3281548,https://www.rottentomatoes.com/m/little_women_...,0.843571
5,Marriage Story,"Noah Baumbach, David Heyman",https://www.imdb.com//title/tt7653254/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,93,8.1,23.0,...,137,0.0,-1.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt7653254,https://www.rottentomatoes.com/m/marriage_stor...,0.851186
6,Once Upon a Time... in Hollywood,"David Heyman, Shannon McIntosh, Quentin Tarantino",https://www.imdb.com//title/tt7131622/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,83,7.7,10.0,...,161,0.0,0.0,-1.0,1.0,1.0,-1.0,"February 9, 2020",tt7131622,https://www.rottentomatoes.com/m/once_upon_a_t...,0.794190
7,Parasite,"Sin-ae Kwak, Bong Joon Ho",https://www.imdb.com//title/tt6751668/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,96,8.6,3.0,...,132,1.0,0.0,-1.0,0.0,-1.0,-1.0,"February 9, 2020",tt6751668,https://www.rottentomatoes.com/m/parasite_2019,0.906155
8,The Irishman,"Martin Scorsese, Robert De Niro, Jane Rosentha...",https://www.imdb.com//title/tt1302006/?ref_=ev...,Academy Awards,2020,Best Motion Picture of the Year,0,1,94,8.0,15.0,...,209,0.0,0.0,-1.0,0.0,0.0,-1.0,"February 9, 2020",tt1302006,https://www.rottentomatoes.com/m/the_irishman_...,0.864167
9,Green Book,"Jim Burke, Charles B. Wessler, Brian Hayes Cur...",https://www.imdb.com//title/tt6966692/?ref_=ev...,Academy Awards,2019,Best Motion Picture of the Year,1,2,69,8.2,111.0,...,130,0.0,0.0,-1.0,0.0,1.0,-1.0,"February 24, 2019",tt6966692,https://www.rottentomatoes.com/m/green_book,0.732727


In [80]:
oscar_picture1.to_csv('4.csv')